## 8.3.1 什么是装饰器

## 8.3.2 基础知识

In [ ]:
# 装饰器可以这样理解，就是直接在函数里面插入一段函数
# func(6,7)
# decorator(func)(6,7)  这个就是装饰器代码了

In [1]:
# 装饰器支持嵌套，也就是支持加入多个装饰器
def d1(F): return lambda: 'X' + F()
def d2(F): return lambda: 'Y' + F()
def d3(F): return lambda: 'Z' + F()

@d1
@d2
@d3
def func():
    return 'spam'

In [2]:
print(func())

XYZspam


In [ ]:
# 装饰器支持添加参数

## 8.3.3 编写函数装饰器

In [3]:
# 首先我们定义了一个装饰器
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
    def __call__(self, *args):
        self.calls += 1
        print('call %s  to %s' % (self.calls, self.func.__name__))
        self.func(*args)

In [6]:
# 我们的装饰器结果如下
@tracer
def spam(a,b,c):
    print(a+b+c)
spam(4,5,6)
spam(7,8,9)

call 1  to spam
15
call 2  to spam
24


In [7]:
# 我们可以直接看到这个函数的参数
print(spam.calls)
print(spam)

2


In [8]:
# 我们可以使用下面的方法来模拟装饰器
calls = 0
def tracer(func,*args):
    global calls
    calls += 1
    print('call %s  to %s' % (calls, func.__name__))
    func(*args)
def spam(a,b,c):
    print(a+b+c)

tracer(spam,1,2,3)
tracer(spam,4,5,6)

call 1  to spam
6
call 2  to spam
15


In [11]:
# 每个函数的装饰器的变量都是单独的
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
    # 函数调用支持位置参数和key类型的参数
    def __call__(self, *args, **kwargs):
        self.calls += 1
        print('call %s  to %s' % (self.calls, self.func.__name__))
        self.func(*args,**kwargs)
@tracer
def spam(a,b,c):
    print(a+b+c)
@tracer
def eggs(x,y):
    print(x**y)

spam(1,2,3)
spam(a=4,b=5,c=6)
print("======")
eggs(2,16)
eggs(4,y=4)

call 1  to spam
6
call 2  to spam
15
call 1  to eggs
65536
call 2  to eggs
256


In [12]:
# 因为类属性不能全局共享，我们可以返回一个包装器
calls = 0
def tracer(func):
    def wrapper(*args, **kwargs):
        global calls
        calls += 1
        print('call %s to %s' % (calls, func.__name__))
        return func(*args,**kwargs)
    return wrapper

@tracer
def spam(a,b,c):
    print(a+b+c)
@tracer
def eggs(x,y):
    print(x**y)
spam(1,2,3)
spam(a=4,b=5,c=6)
print("======")
eggs(2,16)
eggs(4,y=4)

call 1 to spam
6
call 2 to spam
15
call 3 to eggs
65536
call 4 to eggs
256


In [13]:
# 如果还是想让每个函数都有自己的计数器的话
def tracer(func):
    calls = 0
    def wrapper(*args,**kwargs):
        nonlocal calls
        calls += 1
        print('call %s to %s' % (calls, func.__name__))
        return func(*args,**kwargs)
    return wrapper
@tracer
def spam(a,b,c):
    print(a+b+c)
@tracer
def eggs(x,y):
    print(x**y)
spam(1,2,3)
spam(a=4,b=5,c=6)
print("======")
eggs(2,16)
eggs(4,y=4)

call 1 to spam
6
call 2 to spam
15
call 1 to eggs
65536
call 2 to eggs
256


In [1]:
# 或者直接这样给函数加上参数
def tracer(func):
    def wrapper(*args,**kwargs):
        wrapper.calls += 1
        print('call %s to %s' % (wrapper.calls, func.__name__))
        return func(*args,**kwargs)
    wrapper.calls = 0
    return wrapper
@tracer
def spam(a,b,c):
    print(a+b+c)
@tracer
def eggs(x,y):
    print(x**y)
spam(1,2,3)
spam(a=4,b=5,c=6)
print("======")
eggs(2,16)
eggs(4,y=4)

call 1 to spam
6
call 2 to spam
15
call 1 to eggs
65536
call 2 to eggs
256


In [2]:
# 我们可以给装饰器添加描述符
class tracer(object):
    def __init__(self, func):
        self.calls = 0
        self.func = func
    def __call__(self, *args, **kwargs):
        self.calls += 1
        print('call %s  to %s' % (self.calls, self.func.__name__))
        self.func(*args,**kwargs)
    def __get__(self, instance, owner):
        def wrapper(*args,**kwargs):
            print('call get', instance, owner, *args, **kwargs)
            return self(instance, *args, **kwargs)
        return wrapper()

class Person:
    @tracer
    def giveRaise(self):
        return 10

p = Person()
print(p.giveRaise)

call get <__main__.Person object at 0x000001C05AED54C8> <class '__main__.Person'>
call 1  to giveRaise
None


In [3]:
# 上面这个不能适用于函数
@tracer
def spam(a,b,c):
    return a+b+c
print(spam(1,2,3))

call 1  to spam
None


In [4]:
# 我们可以使用装饰器来计时
import time,sys
class timer:
    def __init__(self, func):
        self.func = func
        self.alltime = 0
    def __call__(self, *args, **kwargs):
        start = time.clock()
        result = self.func(*args,**kwargs)
        elapsed = time.clock() - start
        self.alltime += elapsed
        print('%s:%.5f,%.5f' % (self.func.__name__, elapsed, self.alltime))
        return result
@timer
def listcomp(N):
    return [x*3 for x in range(N)]

# 这样我们就可以获取每次计算的耗时信息了
listcomp(50000)
listcomp(50000)
listcomp(50000)
listcomp(50000)
print("all time", listcomp.alltime)

listcomp:0.00718,0.00718
listcomp:0.00620,0.01338
listcomp:0.00597,0.01935
listcomp:0.00622,0.02557
all time 0.02556820000040716


d:\app\python\377\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
d:\app\python\377\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # Remove the CWD from sys.path while we load stuff.


In [8]:
# 最后我们可以给装饰器添加参数
def timer2(label=''):
    def decorator(func):
        def onCall(*args):
            func(*args)
            print(label)
        return onCall
    return decorator

@timer2("--->")
def listLoop(N):
    return [x*3 for x in range(N)]

listLoop(1000)

--->


In [10]:
# 下面我们来改造一下，支持传入参数
def timer(label='',trace=True):
    class Timer:
        def __init__(self, func):
            self.func = func
            self.alltime = 0
        def __call__(self, *args, **kwargs):
            start = time.clock()
            result = self.func(*args,**kwargs)
            elapsed = time.clock() - start
            self.alltime += elapsed
            if trace:
                print('%s %s:%.5f,%.5f' % (label,self.func.__name__, elapsed, self.alltime))
            return result
    return Timer

In [11]:
@timer(label="[CCC]==>")
def listcomp(N):
    return [x*3 for x in range(N)]

listcomp(1000)
listcomp(1000)
listcomp(1000)
listcomp(1000)
print("alll time", listcomp.alltime)

[CCC]==> listcomp:0.00018,0.00018
[CCC]==> listcomp:0.00010,0.00029
[CCC]==> listcomp:0.00005,0.00034
[CCC]==> listcomp:0.00006,0.00040
alll time 0.00040140000010069343


d:\app\python\377\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
d:\app\python\377\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # Remove the CWD from sys.path while we load stuff.


## 8.3.4 编写类装饰器

In [12]:
# 首先我们来创建一个单例类
instances = {}
def singleton(aClass):
    def onCall(*args,**kwargs):
        if aClass not in instances:
            instances[aClass] = aClass(*args,**kwargs)
        return instances[aClass]
    return onCall

In [14]:
# 下面我们我们给类加上这个装饰器
@singleton
class Person:
    def __init__(self,name,hours,rate):
        self.name = name
        self.hours = hours
        self.rate = rate
    def pay(self):
        return self.hours*self.rate

@singleton
class Spam:
    def __init__(self, val):
        self.attr = val

bob = Person('Bob', 40, 10)
print(bob.name, bob.pay())
# 这里我们再创建其实就还是只相当于调用了第一个实例
sue = Person('Sue',50,20)
print(sue.name, sue.pay())

Bob 400
Bob 400


In [15]:
# 其他类也是一样的
x=Spam(42)
y=Spam(99)
print(x.attr, y.attr)

42 42


In [16]:
# 下面我们使用类装饰器来跟踪接口
def Tracer(aClass):
    class Wrapper:
        def __init__(self,*args, **kwargs):
            self.fetches = 0
            self.wrapped = aClass(*args,**kwargs)
        def __getattr__(self, item):
            print('trace:'+item)
            self.fetches += 1
            return  getattr(self.wrapped, item)
    return Wrapper

@Tracer
class Spam:
    def display(self):
        print('spam!'*8)

@Tracer
class Person:
    def __init__(self,name,hours,rate):
        self.name = name
        self.hours = hours
        self.rate = rate
    def pay(self):
        return self.hours * self.rate

food = Spam()
food.display()
print([food.fetches])

bob = Person('bob',40,50)
print(bob.name)
print(bob.pay())

trace:display
spam!spam!spam!spam!spam!spam!spam!spam!
[1]
trace:name
bob
trace:pay
2000


In [17]:
sue = Person('sue',rate=100,hours=60)
print(sue.name)
print(sue.pay())

trace:name
sue
trace:pay
6000


In [18]:
print([bob.fetches,sue.fetches])

[6, 4]


In [ ]:
# 我们也可以给内置类型加上装饰器

## 8.3.5 直接管理函数和类

In [22]:
# 我们可以使用装饰器来管理函数和类
registry = {}
def register(obj):
    registry[obj.__name__] = obj
    return obj
@register
def spam(x):
    return x**2

@register
def ham(x):
    return x**3

@register
class Eggs:
    def __init__(self,x ):
        self.data = x**4
    def __str__(self):
        return str(self.data)

print('register')
for name in registry:
    print(name,'=>',registry[name],type(registry[name]))

print('\nManual calls:')
print(spam(2))
print(ham(2))
x = Eggs(2)
print(x)

print('\nRegistry calls:')
for name in registry:
    print(name,'=>',registry[name](2))

register
spam => <function spam at 0x000001C059537AF8> <class 'function'>
ham => <function ham at 0x000001C059537708> <class 'function'>
Eggs => <class '__main__.Eggs'> <class 'type'>

Manual calls:
4
8
16

Registry calls:
spam => 4
ham => 8
Eggs => 16


## 8.3.6 示例：私有和公有属性

In [23]:
# 这块就是利用装饰器来实现一个私有属性和公有属性的功能，跳过

## 8.3.7 验证函数参数

In [ ]:
# 这个就是使用装饰器来实现一个参数验证的功能，跳过